<a href="https://colab.research.google.com/github/selinakgun/-/blob/master/Selin_Akgun_SayisalGoruntuIsleme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Veri Yükleme**


**1.1. Kütüphanelerin İçe Aktarılması**

In [ ]:
# Temel Veri İşleme Kütüphaneleri
import pandas as pd # Veri Çerçeveleri için
import numpy as np  # Sayısal işlemler ve Görüntü Dizileri için
import os           # Dosya ve Dizin işlemleri için

# Görüntü İşleme Kütüphanesi
# OpenCV (cv2) önerilir, çünkü sonraki adımlarda (renk uzayı dönüşümü, blur vb.) çok işe yarayacaktır.
import cv2          # Görüntü işleme fonksiyonları için
# from PIL import Image # Alternatif olarak PIL/Pillow da kullanılabilir

# Görselleştirme Kütüphaneleri
import matplotlib.pyplot as plt # Grafik ve görselleştirme için
import seaborn as sns           # İstatistiksel görselleştirmeler (özellikle dağılım grafikleri/boyut analizi için) için

print("Gerekli kütüphaneler başarıyla içe aktarıldı.")

Veri Yönetimi ve Temel Hesaplamalar: pandas ve numpy kütüphaneleri, dosya yollarını ve meta verileri içeren DataFrame'i oluşturmak ve görüntülerin piksel değerleri üzerinde hızlı ve etkin matematiksel/sayısal işlemler (ortalama, standart sapma, piksel manipülasyonları) yapmak için kritik öneme sahiptir.

Görüntü İşleme Çekirdeği: cv2 (OpenCV) kütüphanesinin seçilmesi, görüntüyü okuma/yazma, RGB $\rightarrow$ Grayscale dönüşümü, histogram eşitleme, filtreleme (blur), geometrik dönüşümler ve renk uzayı işlemleri (YCrCb) gibi Sayısal Görüntü İşleme ödevinin tüm zorunlu işlemlerini kolaylıkla yapmamızı sağlayacaktır.

Görselleştirme: matplotlib ve seaborn, özellikle görüntülerin orijinal ve işlenmiş hallerini yan yana gösterme, istatistiksel dağılımları (boyut, dosya) analiz etme ve histogramları (RGB ve Grayscale) çizme gereklilikleri için vazgeçilmezdir.

Dosya Yönetimi: os modülü ise, veri setindeki görüntü dosyalarının yollarını dinamik olarak bulup okumak için gereklidir, bu da veri setinin Colab ortamına sorunsuz entegrasyonunu sağlar. 9Bu kütüphane seçimi, hem veri setini yönetmek hem de istenen karmaşık görüntü işleme ve analiz görevlerini (Kontrast Germe, Histogram Eşitleme, Gamma Düzeltme, FFT vb.) gerçekleştirmek için doğru ve eksiksiz bir temel oluşturmaktadır.

**1.2. Veri Setinin Yüklenmesi**

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
nodoubttome_skin_cancer9_classesisic_path = kagglehub.dataset_download('nodoubttome/skin-cancer9-classesisic')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Veri setinizin bulunduğu ana klasörün yolu.
BASE_DIR = '/kaggle/input/skin-cancer9-classesisic/'

image_paths = []

# os.walk'u BASE_DIR'den başlatarak, TAHMİN EDİLEN tüm alt klasörleri taranıyor.
# Bu sayede 'skin cancer ISIC The International Skin Imaging Collaboration' klasörünün içindekileri bulacağız.
print(f"'{BASE_DIR}' altındaki TÜM alt klasörler taranıyor...")

# Klasördeki tüm dosyaları gezme
for dirname, _, filenames in os.walk(BASE_DIR):
    for filename in filenames:
        # JPG, JPEG ve PNG dosyalarını al
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            full_path = os.path.join(dirname, filename)
            image_paths.append(full_path)

# 1. train_df adlı bir DataFrame'e yükleme
# Not: Bu, aslında tüm veri setindeki görüntüleri (Train ve Test) içerecektir.
train_df = pd.DataFrame({
    'filepath': image_paths,
    'filename': [os.path.basename(p) for p in image_paths]
})

# 2. İlk birkaç satırı görüntüleme
print("\n--- train_df İlk 5 Satır ---")
print(train_df.head())

# 3. Toplam görüntü sayısını ekrana yazdırma
print(f"\n✅ Başarıyla Yüklenen Toplam Görüntü Sayısı: {len(train_df)}")

# Eğer görüntüler sadece Train klasöründeyse, DataFrame'i filtreleyebiliriz
train_df = train_df[train_df['filepath'].str.contains('Train', case=False, na=False)].reset_index(drop=True)
print(f"\n✅ 'Train' klasörü filtrelendikten sonraki Toplam Görüntü Sayısı: {len(train_df)}")

Bu adımda, ödev için belirlenen ISIC 2018 Skin Lesion Dataset'inin Kaggle sürümü başarıyla entegre edilmiştir.

Veri Yolu Taraması: os.walk fonksiyonu, veri setinin kök dizini (/kaggle/input/skin-cancer9-classesisic/) altında bulunan tüm alt klasörleri tarayarak toplam 2357 adet görüntü dosyası yolunu toplamıştır.

DataFrame Oluşturma: Toplanan dosya yolları kullanılarak train_df adlı bir Pandas DataFrame'i oluşturulmuştur.

Filtreleme: train_df, yalnızca eğitim (Train) klasöründen gelen dosyaları içerecek şekilde filtrelenmiştir. Bu filtreleme sonucunda, 2239 adet görüntü dosyasının yolu DataFrame'e kaydedilmiştir.

Sonuç: Veri setine erişim sağlanmış, görüntü dosyası yolları bir DataFrame yapısı içinde organize edilmiş ve ilerideki işlemler için temel veri kaynağı hazır hale getirilmiştir. Bu sayı, ödevin sonraki adımları için kullanılacak RGB ve Grayscale dönüşümlerinin temelini oluşturacaktır.

**1.3. Veri Özelliklerinin İncelenmesi**

In [ ]:
# DataFrame'e yükseklik, genişlik ve kanal bilgilerini eklemek için fonksiyon
def get_image_properties(filepath):
    try:
        # cv2 ile görüntüyü yükle
        img = cv2.imread(filepath)

        if img is None:
             return 0, 0, 0, 0

        height, width = img.shape[:2]
        # Kanal sayısını al (len(img.shape) == 3 ise 3, aksi takdirde 1)
        channels = img.shape[2] if len(img.shape) == 3 else 1

        # Dosya boyutunu os modülü ile al
        file_size = os.path.getsize(filepath)

        return height, width, channels, file_size

    except Exception as e:
        # Hata durumunda (örn. bozuk dosya)
        print(f"Hata oluştu: {filepath}: {e}")
        return 0, 0, 0, 0

print("\n--- Görüntü Özelliklerinin Analizi Başlıyor ---")

# Özellikleri DataFrame'e uygulama (Bu adım biraz zaman alabilir)
# train_df'e yeni sütunlar ekleniyor
train_df[['height', 'width', 'channels', 'file_size']] = train_df['filepath'].apply(
    lambda x: pd.Series(get_image_properties(x))
)

# Hata veren/yüklenemeyen satırları temizleme (boyut 0 olanlar)
train_df = train_df[train_df['height'] > 0].reset_index(drop=True)
print(f"✅ Analiz için kullanılan toplam geçerli görüntü sayısı: {len(train_df)}")

# --- 1. Çözünürlük (Yükseklik–Genişlik) Analizi ---
print("\n--- 1. Çözünürlük Analizi ---")
resolution_stats = train_df[['height', 'width']].describe()
print(resolution_stats)

# Çözünürlük dağılımını görselleştirme
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(train_df['height'], kde=True, bins=50)
plt.title('Yükseklik Dağılımı')
plt.subplot(1, 2, 2)
sns.histplot(train_df['width'], kde=True, bins=50)
plt.title('Genişlik Dağılımı')
plt.tight_layout()
plt.show()

#


# --- 2. Kanal Sayısı Doğrulama ---
print("\n--- 2. Kanal Sayısı Doğrulama ---")
channel_counts = train_df['channels'].value_counts()
print("Kanal Sayısı Dağılımı:\n", channel_counts)

if 3 in channel_counts and channel_counts[3] == len(train_df):
    print("✅ Tüm görüntüler 3 kanallıdır (RGB).")
else:
    print(f"⚠️ Farklı kanal sayıları mevcut. En yaygın olanı {channel_counts.index[0]} kanal.")


# --- 3. Dosya Boyutu Dağılımı ---
print("\n--- 3. Dosya Boyutu (MB) Analizi ---")
# Byte'tan Megabyte'a dönüştürme
train_df['file_size_mb'] = train_df['file_size'] / (1024 * 1024)
file_size_stats = train_df['file_size_mb'].describe()
print(file_size_stats)

# Dosya boyutu dağılımını görselleştirme
plt.figure(figsize=(6, 5))
sns.histplot(train_df['file_size_mb'], bins=50, kde=True)
plt.title('Dosya Boyutu Dağılımı (MB)')
plt.xlabel('Dosya Boyutu (MB)')
plt.show()

#

Bu adımda, eğitim veri setinde bulunan 2239 geçerli görüntü üzerinde detaylı bir analiz gerçekleştirilmiştir.

**1. Çözünürlük (Yükseklik–Genişlik) Analizi**

Görüntülerin ortalama çözünürlüğü yaklaşık $651 \times 862$ pikseldir. Ancak, standart sapma değerlerinin (Yükseklik için $\sim 430$, Genişlik için $\sim 576$) ortalamaya göre oldukça yüksek olması, veri setindeki çözünürlüklerin çok geniş bir aralığa (Minimum $450 \times 576$ ile Maksimum $2592 \times 3872$ piksel arasında) yayıldığını göstermektedir.Histogramlar, görüntülerin büyük çoğunluğunun minimum boyutlara yakın (özellikle $450-800$ piksel) yoğunlaştığını, ancak aynı zamanda önemli sayıda görüntünün $2000$ pikselden yüksek çözünürlüklere sahip olduğunu göstermektedir. Bu yüksek varyasyon, ilerideki tüm görüntü işleme (özellikle filtreleme ve döndürme gibi matris işlemleri) adımlarından önce boyut standardizasyonunun (yeniden boyutlandırma) yapılmasını kritik ve zorunlu kılmaktadır.

**2. Kanal Sayısı Doğrulama**

Kanal Sayısı Dağılımı:

3 Kanal: 2239 görüntü

Yorum: Yapılan analiz sonucunda, veri setindeki tüm 2239 görüntünün 3 kanallı (RGB) olduğu doğrulanmıştır. Bu, ödevde belirtilen hem RGB (renkli) hem de Grayscale (gri tonlamalı) işlemlerinin, başlangıçtaki renkli veriden türetileceğini kesinleştirmektedir.

**3. Dosya Boyutu Dağılımı**


Dosya boyutları ortalama $\sim 0.28$ MB civarındadır ve $0.023$ MB ile $3.668$ MB arasında geniş bir aralıkta değişim göstermektedir. Histogram, dosyaların büyük çoğunluğunun $0.5$ MB'ın altında yoğunlaştığını gösterirken, yüksek boyuttaki ($\sim 3.6$ MB) uç değerler, bazı görüntülerin çözünürlüklerinin yüksek olmasından veya daha az sıkıştırma ile saklanmasından kaynaklandığını düşündürmektedir. Bu dağılım, veri setinin bellekte (RAM) etkin bir şekilde işlenebilmesi için optimizasyon gerektirebileceğini göstermektedir.

# **2. Görüntü Yükleme ve Görselleştirme**

**2.1. Rastgele Görüntüler Seçme**

In [ ]:
# 9 rastgele görüntü seçme
random_samples = train_df.sample(n=9, random_state=42).reset_index(drop=True)
# reset_index(drop=True) ekledik, böylece i değerleri 0'dan 8'e gider

# Görselleştirme için subplot oluşturma: 9 satır, her satırda RGB ve Grayscale olmak üzere 2 sütun
fig, axes = plt.subplots(nrows=9, ncols=2, figsize=(10, 35))
fig.suptitle('Rastgele Seçilen 9 Görüntü: RGB ve Grayscale Karşılaştırması', y=1.02)

# i yerine idx (satır sayacını) kullanıyoruz. idx: 0'dan 8'e gider
for idx, row in random_samples.iterrows():
    filepath = row['filepath']

    # 1. Görüntüyü Yükle ve BGR -> RGB/Grayscale Dönüşümü
    img_bgr = cv2.imread(filepath)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    height, width = img_rgb.shape[:2]

    # --- RGB Görselleştirme ---
    # axes'in idx. satırının 0. sütununa yerleştir
    axes[idx, 0].imshow(img_rgb)
    axes[idx, 0].set_title(f"RGB ({height}x{width})")
    axes[idx, 0].axis('off')

    # --- Grayscale Görselleştirme ---
    # axes'in idx. satırının 1. sütununa yerleştir
    axes[idx, 1].imshow(img_gray, cmap='gray')
    axes[idx, 1].set_title(f"Grayscale ({height}x{width})")
    axes[idx, 1].axis('off')

plt.tight_layout(rect=[0, 0, 1, 1.01])
plt.show()

Rastgele seçilen 9 deri lezyonu görüntüsünün RGB ve Grayscale formatları yan yana görselleştirilmiştir.

RGB Görüntü Analizi
Renk Bilgisi: RGB formatı, dermatolojik görüntülerde kritik olan üç temel renk bilgisini (Kırmızı, Yeşil, Mavi) korumuştur. Bu, lezyonun pigmentasyonunun (kahverengi/siyah) ve çevresindeki kızarıklığın (eritem) veya damar yapılarının (kırmızı/pembe tonlar) net bir şekilde ayrılmasına olanak tanır.

Dokusal Zenginlik: RGB formatı, kıl, hava kabarcıkları veya dermatopatolojik özellikler gibi ince dokusal ayrıntıları ve yüzey anomalilerini en zengin haliyle sunar.

Grayscale Görüntü Analizi
Yoğunluk Dönüşümü: Grayscale'e dönüşüm, renk bilgisini tamamen ortadan kaldırmış ve görüntüyü yalnızca parlaklık yoğunluğu (luminance) matrisine indirgemiştir.

Kontrast ve Sınırlar: Lezyonlar, çevrelerindeki sağlıklı deriye göre koyu tonlarda (düşük piksel değerlerinde) belirginleşmiştir. Bu dönüşüm, özellikle lezyon sınırlarının belirgin olduğu veya düşük ışıkta çekilmiş görüntülerde kontrastı vurgulamaya yardımcı olabilir.

Bilgi Kaybı: Dönüşüm, renk bilgisine dayalı ayrımların kaybolmasına neden olmuştur. Örneğin, kırmızı (eritem) ve kahverengi (pigmentasyon) tonlarındaki benzer parlaklığa sahip alanlar, Grayscale görüntüde aynı gri tonlarda görünebilir, bu da renk tabanlı ayrımı zorlaştırır.

RGB ve Grayscale Karşılaştırması
Uygulama Alanı: RGB formatı, renk uzayı manipülasyonları, renk histogramları ve teşhise yönelik özellik çıkarımı için temel format olmaya devam ederken; Grayscale formatı, kenar tespiti, Frekans Alanında Filtreleme (FFT) ve tek kanallı gürültü azaltma (blur) gibi uzamsal işlemlerde karmaşıklığı azaltmak için idealdir.

Piksel Değeri: Grayscale görüntüler, RGB görüntünün üç kanalının ağırlıklı ortalamasını temsil ettiği için, piksel değerleri (parlaklık) açısından RGB'nin toplam parlaklığını temsil eder.

**2.2. Rastgele Görüntülerin İstatistiksel Özellikleri**

In [ ]:
# Gerekli kütüphaneleri içe aktar
import numpy as np
import pandas as pd
import cv2
import os

# Not: train_df'in daha önceki adımlarda doğru şekilde yüklendiği ve 'filepath' sütununu içerdiği varsayılmaktadır.

# 2.1 adımında seçilen 9 görüntüyü tekrar alıyoruz (Tekrar üretilebilir olması için aynı random_state'i kullanın)
# Eğer train_df'te bu görüntüler yoksa veya yüklenemezse hata oluşur.
try:
    random_samples = train_df.sample(n=9, random_state=42).reset_index(drop=True)
except NameError:
    print("Hata: train_df DataFrame'i bulunamadı. Lütfen 1.2 ve 1.3 adımlarını tekrar çalıştırın.")
    # Örnek bir boş DataFrame oluşturup çıkıyoruz.
    random_samples = pd.DataFrame({'filepath': []})

if len(random_samples) == 0:
    print("Hata: random_samples boş. Devam edilemiyor.")

# Sonuçları tutmak için bir liste
results = []

print("--- 2.2. Piksel İstatistikleri Karşılaştırması Başlıyor ---")

# Görüntüler üzerinde döngü
for idx, row in random_samples.iterrows():
    filepath = row['filepath']

    # Görüntüyü BGR olarak yükle
    img_bgr = cv2.imread(filepath)

    # Hata kontrolü
    if img_bgr is None:
        print(f"Uyarı: Görüntü yüklenemedi: {filepath}")
        continue

    # BGR'dan RGB'ye dönüştür
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # Grayscale'e dönüştür
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)


    # --- RGB İstatistikleri ---
    # RGB görüntü 3 boyutludur. Tüm pikselleri tek bir diziye yayarak hesapla.
    rgb_min = np.min(img_rgb)
    rgb_max = np.max(img_rgb)
    rgb_mean = np.mean(img_rgb)
    rgb_std = np.std(img_rgb)

    # --- Grayscale İstatistikleri ---
    # Grayscale görüntü 2 boyutludur.
    gray_min = np.min(img_gray)
    gray_max = np.max(img_gray)
    gray_mean = np.mean(img_gray)
    gray_std = np.std(img_gray)

    # Sonuçları kaydet
    results.append({
        'Görüntü No': idx + 1,
        'Format': 'RGB',
        'Min': rgb_min,
        'Max': rgb_max,
        'Ortalama': f"{rgb_mean:.2f}",
        'Std Sapma': f"{rgb_std:.2f}"
    })
    results.append({
        'Görüntü No': idx + 1,
        'Format': 'Grayscale',
        'Min': gray_min,
        'Max': gray_max,
        'Ortalama': f"{gray_mean:.2f}",
        'Std Sapma': f"{gray_std:.2f}"
    })

# Sonuçları DataFrame'e dönüştürme ve gösterme
stats_df = pd.DataFrame(results)

# DataFrame'i CSV'ye yaz
stats_csv_path = 'piksel_istatistikleri.csv'
stats_df.to_csv(stats_csv_path, index=False)

print(f"\nİstatistikler '{stats_csv_path}' dosyasına kaydedildi.")
print("\n--- 2.2. Piksel İstatistikleri Karşılaştırması Tablosu ---")
print(stats_df.to_markdown(index=False))

**1. Min, Max Değerler ve Dinamik AralıkMin Değerler:**
Çoğu görüntüde Min değer 0 olarak gözlemlenmiştir (Örn: Görüntü No 1, 4, 5, 9 RGB). Bu, görüntülerin tamamen siyah (0) pikseller içerdiğini gösterir. Ancak, bazı Grayscale görüntülerde minimum değerin yüksek olduğu görülmektedir (Örn: Görüntü No 3 Grayscale: 82, Görüntü No 6 Grayscale: 98).Max Değerler: Maksimum değer, çoğu görüntüde 240 ile 255 arasında değişmektedir. Maksimum piksel değerinin 255 olmaması (Örn: Görüntü No 1 RGB: 240), görüntünün en parlak beyaz pikselleri bile tam parlaklığa (255) ulaşmadığını gösterir.Sonuç: Min ve Max değerlerinin $\mathbf{0}$ ve $\mathbf{255}$ aralığını tam olarak kullanmaması (Örn: Görüntü No 3 Grayscale: $\mathbf{82}$ Min, $\mathbf{230}$ Max), bu görüntülerin düşük kontrasta sahip olduğunu ve Kontrast Germe (Stretching) işleminden fayda göreceğini işaret eder.

**2. Ortalama Değerler Analizi (Parlaklık)Ortalama Aralığı:** Ortalama piksel değerleri $136.53$ (Görüntü No 9 RGB) ile $193.71$ (Görüntü No 8 RGB) arasında değişmektedir. Bu, görüntülerimizin genellikle orta-yüksek parlaklık aralığında olduğunu, ancak bazı lezyonların (No 9 gibi) genel olarak daha koyu olduğunu gösterir.RGB vs Grayscale: Her görüntü çiftinde, RGB Ortalama değeri ile Grayscale Ortalama değeri birbirine çok yakındır (Örn: Görüntü No 4 RGB: $149.82$ vs Grayscale: $146.27$). Bu durum, Grayscale dönüşümünün görüntünün genel parlaklık algısını veya aydınlatma bilgisini büyük ölçüde koruduğunu teyit eder.

**3. Standart Sapma (Std Sapma) Karşılaştırması (Kontrast):**
Standart sapma (Std Sapma), piksel değerlerinin ortalamadan ne kadar yayıldığını, yani görüntüdeki kontrastı ve yoğunluk çeşitliliğini gösterir.

Grayscale Std Sapma $\le$ RGB Std Sapma: Tüm görüntülerde, Grayscale formatının standart sapması, karşılık gelen RGB formatının sapmasından daha düşüktür (Örn: Görüntü No 2 RGB: $36.80$ vs Grayscale: $32.33$).

Neden: RGB standardı sapması, tüm üç kanalın birleşik varyasyonunu ölçerken; Grayscale dönüşümü, renk bilgisini birleştirerek varyasyonu tek bir parlaklık kanalında toplar, bu da toplam varyansın azalmasına neden olur.

Düşük Kontrast Durumları: Görüntü No 6'da Grayscale Std Sapma'nın sadece $12.93$ olması, bu görüntünün çok düşük kontrasta sahip olduğunu ve piksel değerlerinin birbirine aşırı derecede yakın olduğunu gösterir. Bu görüntü, kontrast artırıcı işlemler için en uygun adaylardan biridir.

**2.3. Histogram Çizimi (Zorunlu – RGB + Grayscale)**

**a) RGB Histogramı**

In [ ]:
# Gerekli kütüphaneleri içe aktar
import numpy as np
import pandas as pd
from PIL import Image
import os
import matplotlib.pyplot as plt

# --- 1. Görüntüyü Seçme ve Yükleme ---
# 2.1'de kullandığımız aynı random_state'i kullanarak 9 görüntüyü tekrar seçiyoruz.
random_samples = train_df.sample(n=9, random_state=42).reset_index(drop=True)

# Analiz için Görüntü No 6'yı seçelim (idx = 5). Kontrastı en düşük olanlardan biriydi.
IMAGE_INDEX_TO_ANALYZE = 5
filepath = random_samples.loc[IMAGE_INDEX_TO_ANALYZE, 'filepath']

try:
    img_pil = Image.open(filepath)
    # NumPy dizilerine dönüştürme
    img_rgb = np.array(img_pil)
    img_gray_pil = img_pil.convert('L')
    img_gray = np.array(img_gray_pil)
except Exception as e:
    print(f"Hata: Görüntü yüklenemedi veya dönüştürülemedi: {e}")
    exit()

# --- 2. RGB ve Grayscale Histogramlarını Çizme ---

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))
fig.suptitle(f'Görüntü No {IMAGE_INDEX_TO_ANALYZE + 1} Histogram Analizi (Piksel Yoğunluğu 0-255)', fontsize=14)

# A) RGB Histogramı (R, G, B kanallarını ayrı ayrı tek grafikte)
colors = ('r', 'g', 'b')
channel_names = ('Kırmızı (R)', 'Yeşil (G)', 'Mavi (B)')

for i, color in enumerate(colors):
    # numpy'ı kullanarak histogramı hesapla
    hist, bins = np.histogram(img_rgb[:, :, i].flatten(), bins=256, range=[0, 256])

    # Histogramı çiz
    axes[0].plot(bins[:-1], hist, color=color, alpha=0.7, label=channel_names[i])

axes[0].set_title('a) RGB Histogramı (Üç Kanal)')
axes[0].set_xlabel('Piksel Yoğunluğu')
axes[0].set_ylabel('Piksel Sayısı')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.5)


# B) Grayscale Histogramı (Tek Kanal)
hist_gray, bins_gray = np.histogram(img_gray.flatten(), bins=256, range=[0, 256])

# Histogramı çiz
axes[1].plot(bins_gray[:-1], hist_gray, color='black', alpha=0.9)
axes[1].fill_between(bins_gray[:-1], hist_gray, color='gray', alpha=0.3)


axes[1].set_title('b) Grayscale Histogramı (Tek Kanal)')
axes[1].set_xlabel('Piksel Yoğunluğu')
axes[1].set_ylabel('Piksel Sayısı')
axes[1].grid(axis='y', alpha=0.5)


plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

#

Analiz, kontrastının düşük olduğu düşünülen rastgele seçilmiş bir cilt lezyonu görüntüsü üzerinde gerçekleştirilmiştir (Örneğin, önceki adımda Görüntü No 6).

**a) RGB Histogramı Yorumu (R, G, B Kanalları)**

Piksel Yoğunluk Dağılımı: RGB histogramında görüldüğü gibi, tüm R, G ve B kanallarındaki piksel yoğunluğu $0$ (siyah) ile $255$ (beyaz) aralığının tamamını doldurmamaktadır. Histogram, genellikle $\mathbf{80}$ ile $\mathbf{240}$ piksel yoğunlukları arasında yoğunlaşmıştır. Piksel değerlerinin bu dar alanda toplanması, görüntünün düşük kontrastlı olduğunu açıkça göstermektedir.

Kanal Ayrımı: Kanallar birbirinden belirgin şekillerde ayrılmaktadır:

Kırmızı (R) ve Yeşil (G) kanalları (Genellikle $\sim 100-240$ aralığında) Mavi (B) kanaldan (Genellikle $\sim 80-200$ aralığında) daha yüksek yoğunluklarda (daha sağda) yoğunlaşmıştır.

Bu durum, görüntünün genel olarak kırmızı/kahverengi (yüksek R ve G içeriği) ve parlak bir görünüme sahip olduğunu, Mavi pigmentasyonun (düşük B içeriği) ise baskın olmadığını gösterir.

b) Grayscale Histogramı Yorumu (Tek Kanal)

Yoğunlaşma Alanı: Grayscale histogramı, tek bir geniş pik ile genellikle $\mathbf{100}$ ile $\mathbf{230}$ arasındaki orta-parlak tonlarda yoğunlaşmaktadır. Piklerin bu alanda toplanması, görüntünün genel olarak orta parlaklıkta olduğunu teyit eder.

Dağılımın Darlığı: Grayscale dağılımı da, RGB kanallarının her biri gibi, $0-255$ aralığında dar bir alana sıkışmıştır. Bu darlık, görüntüdeki parlaklık varyasyonunun çok az olduğunu, yani kontrastın düşük olduğunu doğrulamaktadır.

c) RGB $\to$ Grayscale Dönüşümünün Histogram Üzerindeki Etkisi

Birleşme Etkisi: Grayscale histogramı, RGB histogramındaki $\text{R}$, $\text{G}$ ve $\text{B}$ kanallarının dağılımlarının ağırlıklı toplamı (parlaklık formülü) sonucudur.

Varyasyon Kaybı: RGB histogramındaki üç ayrı dağılım (üç farklı renk varyasyonu), Grayscale dönüşümü sonrası tek bir yoğunluk dağılımına dönüşmüştür. Bu, renk bilgisinin kaybolmasıyla birlikte, piksel yoğunluk çeşitliliğinin de tek bir eksende toplanması anlamına gelir.

İyileştirme Sinyali: Hem RGB hem de Grayscale histogramlarının dar bir alanda toplanması, görüntünün dinamik aralığını genişletmek ve kontrastı artırmak için Kontrast Germe (Stretching) ve Histogram Eşitleme (Equalization) gibi iyileştirme tekniklerinin uygulanması gerektiğini açıkça göstermektedir.

# **3. Görüntü İşleme ve İyileştirme**

**3.1. Kontrast Germe (Stretching)**

In [ ]:
# Gerekli kütüphaneleri içe aktar
import numpy as np
import pandas as pd
from PIL import Image
import os
import matplotlib.pyplot as plt

# --- 1. train_df Yeniden Yükleme (Önceki Adımlardan Alındı) ---
BASE_DIR = '/kaggle/input/skin-cancer9-classesisic/'
image_paths = []
for dirname, _, filenames in os.walk(BASE_DIR):
    for filename in filenames:
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_paths.append(os.path.join(dirname, filename))

train_df = pd.DataFrame({'filepath': image_paths})
train_df = train_df[train_df['filepath'].str.contains('Train', case=False, na=False)].reset_index(drop=True)


# --- 2. Kontrast Germe Fonksiyonu Tanımlama ---
def contrast_stretching(img_array):
    """Görüntü üzerindeki minimum ve maksimum piksel değerlerini kullanarak kontrast germe uygular."""

    # Görüntü matrisini float32'ye dönüştürerek hassasiyeti artır
    img_float = img_array.astype(np.float32)

    # Min ve Max değerlerini hesapla
    min_val = np.min(img_float)
    max_val = np.max(img_float)

    # Eğer min ve max eşitse (siyah veya beyaz tek renkli görüntü), işlem yapma
    if max_val == min_val:
        return img_array

    # Kontrast Germe Formülü: I_out = (I_in - Min) * (255 / (Max - Min))
    # Bu formül, tüm değerleri 0-255 aralığına doğrusal olarak yayar
    stretched_img = (img_float - min_val) * (255.0 / (max_val - min_val))

    # Sonuçları uint8 (0-255) tipine geri dönüştür ve sınırla
    return np.clip(stretched_img, 0, 255).astype(np.uint8)

# --- 3. Görüntü Seçimi ve Dönüşümler ---
# 2.3'te analiz edilen Görüntü No 6'yı seçelim (Index 5)
random_samples = train_df.sample(n=9, random_state=42).reset_index(drop=True)
filepath = random_samples.loc[5, 'filepath']

# Yükleme ve dönüştürme (PIL ile RGB -> Grayscale)
img_pil = Image.open(filepath).convert('RGB')
img_rgb_original = np.array(img_pil)
img_gray_original = np.array(img_pil.convert('L'))

# --- 4. Kontrast Germe İşlemleri ---

# RGB Kontrast Germe (Her kanal için ayrı ayrı)
img_rgb_stretched = np.zeros_like(img_rgb_original)
for i in range(3):
    img_rgb_stretched[:, :, i] = contrast_stretching(img_rgb_original[:, :, i])

# Grayscale Kontrast Germe (Tek kanalda)
img_gray_stretched = contrast_stretching(img_gray_original)


# --- 5. Karşılaştırmalı Görselleştirme ---
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))
fig.suptitle('3.1 Kontrast Germe (Stretching) Karşılaştırması', fontsize=16)

# Üst Sol: Orjinal RGB
axes[0, 0].imshow(img_rgb_original)
axes[0, 0].set_title('Orijinal RGB')
axes[0, 0].axis('off')

# Üst Sağ: Kontrastı Gerilmiş RGB
axes[0, 1].imshow(img_rgb_stretched)
axes[0, 1].set_title('Gerilmiş RGB (Kanal Başına)')
axes[0, 1].axis('off')

# Alt Sol: Orjinal Grayscale
axes[1, 0].imshow(img_gray_original, cmap='gray')
axes[1, 0].set_title('Orijinal Grayscale')
axes[1, 0].axis('off')

# Alt Sağ: Kontrastı Gerilmiş Grayscale
axes[1, 1].imshow(img_gray_stretched, cmap='gray')
axes[1, 1].set_title('Gerilmiş Grayscale (Tek Kanal)')
axes[1, 1].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

#

Kontrast germe işlemi, önceki analizlerde (2.2. Piksel İstatistikleri ve 2.3. Histogram Çizimi) düşük kontrasta sahip olduğu belirlenen tek bir temsilci görüntü (Görüntü No 6) üzerinde uygulanmıştır. Bu yöntem, raporlama kolaylığı ve analiz derinliği sağlamak amacıyla seçilmiştir, zira işlem tüm görüntülerde aynı matematiksel prensibe dayanmaktadır.Kontrast Germe EtkisiGenel Etki: İşlem sonrası görüntülerde hem RGB hem de Grayscale formatlarında kontrastın belirgin bir şekilde arttığı gözlemlenmiştir. Soluk veya gri görünen alanlar, artık $0$ (siyah) ile $255$ (beyaz) arasındaki tüm dinamik aralığa yayılmış, daha net ve canlı görünmektedir.Detay Vurgusu: Kontrast artışı sayesinde, lezyonun sınırları, pigmentasyonun koyu noktaları ve çevresindeki ince kıl/damar detayları orijinaline göre çok daha keskin hale gelmiştir. Bu iyileştirme, görsel analizi ve ilerideki segmentasyon adımlarını kolaylaştıracaktır.


<table style="width:100%; border-collapse: collapse;">
    <thead>
        <tr style="background-color: #f2f2f2;">
            <th style="border: 1px solid #ddd; padding: 8px; text-align: left;">Özellik</th>
            <th style="border: 1px solid #ddd; padding: 8px; text-align: left;">RGB Kontrast Germe (Kanal Başına)</th>
            <th style="border: 1px solid #ddd; padding: 8px; text-align: left;">Grayscale Kontrast Germe (Tek Kanal)</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>İşlem Tipi</b></td>
            <td style="border: 1px solid #ddd; padding: 8px;">Germe işlemi, <b>R</b>, <b>G</b> ve <b>B</b> kanallarının her birindeki Min/Max değerleri bağımsız olarak hesaplanarak, her kanala ayrı ayrı uygulanmıştır.</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Germe işlemi, görüntünün tek bir parlaklık kanalı üzerindeki genel Min/Max değerleri hesaplanarak uygulanmıştır.</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Görsel Etki</b></td>
            <td style="border: 1px solid #ddd; padding: 8px;">Kontrast belirgin şekilde artmıştır. Lezyonların çevresindeki <b>renkli doku ve damar yapıları</b> daha canlı ve keskin hale gelmiştir.</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Kontrast belirgin şekilde artmıştır. Sadece <b>parlaklık ve karanlık</b> tonlar arasındaki ayrım keskinleşmiştir, ancak renk bilgisi yoktur.</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Renk Bilgisi</b></td>
            <td style="border: 1px solid #ddd; padding: 8px;">Her kanalın ayrı gerilmesi, orijinal görüntüdeki <b>renk dengesini hafifçe değiştirebilir</b> (renk kayması), ancak genel renk bilgisi korunur.</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Renk bilgisi tamamen ortadan kalkar.</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Kullanım Amacı</b></td>
            <td style="border: 1px solid #ddd; padding: 8px;">Görsel kaliteyi ve lezyonun renk tabanlı özelliklerini koruyarak kontrastı maksimize etmek isteyen <b>insan gözüyle yapılan analizler</b> veya <b>renge duyarlı</b> modeller için uygundur.</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Özellikle <b>kenar tespiti</b>, <b>morfolojik operasyonlar</b> ve <b>frekans alanı filtrelemesi</b> gibi sadece yoğunluk bilgisine ihtiyaç duyan <b>uzamsal</b> işlemler için daha verimli bir ön işlem sağlar.</td>
        </tr>
    </tbody>
</table>
```

```



**3.2. Histogram Eşitleme (Equalization)**

In [ ]:
# Gerekli kütüphaneleri içe aktar (OpenCV'yi kullanmak idealdir)
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt

# --- 1. train_df Yeniden Yükleme ---
# (Önceki adımlarda kullandığınız path'i kullanın)
BASE_DIR = '/kaggle/input/skin-cancer9-classesisic/'
image_paths = []
for dirname, _, filenames in os.walk(BASE_DIR):
    for filename in filenames:
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_paths.append(os.path.join(dirname, filename))

train_df = pd.DataFrame({'filepath': image_paths})
train_df = train_df[train_df['filepath'].str.contains('Train', case=False, na=False)].reset_index(drop=True)


# --- 2. Görüntü Seçimi ve Yükleme ---
# 2.3/3.1'de analiz edilen Görüntü No 6'yı seçelim (Index 5)
random_samples = train_df.sample(n=9, random_state=42).reset_index(drop=True)
filepath = random_samples.loc[5, 'filepath']

# Yükleme (BGR formatında)
img_bgr_original = cv2.imread(filepath)

# Görselleştirme için BGR -> RGB dönüşümü
img_rgb_original = cv2.cvtColor(img_bgr_original, cv2.COLOR_BGR2RGB)

# Grayscale orijinal
img_gray_original = cv2.cvtColor(img_bgr_original, cv2.COLOR_BGR2GRAY)


# --- 3. RGB İçin Histogram Eşitleme ---
# a) RGB → YCrCb dönüşümü
img_ycrcb = cv2.cvtColor(img_bgr_original, cv2.COLOR_BGR2YCrCb)

# b) Sadece Y (Luminance) kanalını ayırma
Y_channel = img_ycrcb[:, :, 0]

# c) Y kanalı üzerinde eşitleme
Y_equalized = cv2.equalizeHist(Y_channel)

# d) Eşitlenmiş Y kanalını geri birleştirme
img_ycrcb[:, :, 0] = Y_equalized

# e) Tekrar RGB’ye dönüştürme (Görselleştirme için)
img_rgb_equalized = cv2.cvtColor(img_ycrcb, cv2.COLOR_YCrCb2RGB)


# --- 4. Grayscale İçin Histogram Eşitleme ---
# Doğrudan histogram eşitleme uygula
img_gray_equalized = cv2.equalizeHist(img_gray_original)


# --- 5. Karşılaştırmalı Görselleştirme ---
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))
fig.suptitle('3.2 Histogram Eşitleme Karşılaştırması', fontsize=16)

# Üst Sol: Orjinal RGB
axes[0, 0].imshow(img_rgb_original)
axes[0, 0].set_title('Orijinal RGB')
axes[0, 0].axis('off')

# Üst Sağ: Eşitlenmiş RGB (Y kanalından)
axes[0, 1].imshow(img_rgb_equalized)
axes[0, 1].set_title('Eşitlenmiş RGB (Y Kanalı)')
axes[0, 1].axis('off')

# Alt Sol: Orjinal Grayscale
axes[1, 0].imshow(img_gray_original, cmap='gray')
axes[1, 0].set_title('Orijinal Grayscale')
axes[1, 0].axis('off')

# Alt Sağ: Eşitlenmiş Grayscale
axes[1, 1].imshow(img_gray_equalized, cmap='gray')
axes[1, 1].set_title('Eşitlenmiş Grayscale')
axes[1, 1].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

#

Histogram eşitleme işlemi, kontrast iyileştirmesi için Kontrast Germe'ye alternatif olarak uygulanmıştır. Uygulama, ödev gereği RGB için renk uzayı dönüşümüyle, Grayscale için ise doğrudan yapılmıştır.

Genel Etki ve Başarı
Kontrast Artışı: Her iki eşitleme yönteminde de (RGB ve Grayscale) görüntü kontrastı, orijinal görüntülere göre belirgin ve agresif bir şekilde artmıştır. Önceden soluk veya ayırt edilemeyen lezyon içi detaylar, artık daha keskin yoğunluk farklarına sahiptir.

Detay Vurgusu: Özellikle lezyonun koyu pigmentasyonlu bölgeleri ve çevresindeki açık cilt dokusu arasındaki sınırlar belirginleşmiştir.



```
# Bu, kod olarak biçimlendirilmiştir
```

<table style="width:100%; border-collapse: collapse; font-family: Arial, sans-serif;">
    <thead>
        <tr style="background-color: #e0f7fa;">
            <th style="border: 1px solid #00bcd4; padding: 10px; text-align: left; color: #000;">Özellik</th>
            <th style="border: 1px solid #00bcd4; padding: 10px; text-align: left; color: #000;">RGB Eşitleme (YCrCb Yöntemi)</th>
            <th style="border: 1px solid #00bcd4; padding: 10px; text-align: left; color: #000;">Grayscale Eşitleme (Doğrudan)</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>İşlem Mekanizması</b></td>
            <td style="border: 1px solid #ddd; padding: 8px;">YCrCb renk uzayına geçiş yapılır; sadece <b>Y (Luminance/Parlaklık)</b> kanalı üzerinde eşitleme uygulanır. Cr ve Cb (Renk) kanalları korunur.</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Görüntünün tek kanallı piksel yoğunluğu üzerinde, renk uzayı dönüşümü olmadan doğrudan eşitleme yapılır.</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Görsel Etki</b></td>
            <td style="border: 1px solid #ddd; padding: 8px;">Kontrast artışı sağlanırken, lezyonun orijinal <b>renk tonları korunur</b>. Sonuç, hem yüksek kontrastlı hem de doğal renklidir (En üstün iyileştirme yöntemi).</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Çok yüksek kontrast elde edilir, ancak tamamen <b>siyah-beyaz</b> (tek renkli) bir görünümdedir.</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Renk Bilgisi</b></td>
            <td style="border: 1px solid #ddd; padding: 8px;">Eşitleme parlaklığa odaklandığı için <b>renk bozulması olmaz</b> ve renk bilgisi korunur.</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Renk bilgisi tamamen kaybolur.</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Gürültü Hassasiyeti</b></td>
            <td style="border: 1px solid #ddd; padding: 8px;">Gürültü artışı, renk kanallarının sabit kalması nedeniyle daha <b>kontrollüdür</b>.</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Dağılımı zorla düzleştirdiği için, görüntünün yapısındaki küçük gürültüyü (hışırtıyı) <b>agresif bir şekilde yükseltme</b> eğilimi gösterir.</td>
        </tr>
    </tbody>
</table>

Histogram Eşitleme, düşük kontrast sorununu çözmede Kontrast Germe kadar etkili bir yöntemdir. Ancak, RGB görüntülerde YCrCb renk uzayını kullanmak, eşitleme işlemini sadece parlaklığa odaklayarak renk bilgisini koruduğu için en üstün iyileştirme yöntemini sunar. Grayscale eşitleme ise, gürültü yükseltme riski taşısa da, tek kanallı işlemlerde hızlı bir kontrast iyileştirme sağlar.

**3.3. Gamma Düzeltme**

Hemen 3.3. Gamma Düzeltme adımına geçelim.Gamma düzeltme, görüntünün parlaklık seviyelerini $\mathbf{doğrusal olmayan}$ bir şekilde ayarlar. Bu, görüntünün orta tonlarını aydınlatmak veya karartmak için Kontrast Germe ve Histogram Eşitleme'den farklı bir kontrol sağlar.$\mathbf{\gamma < 1.0}$ (Örn: $0.5$): Görüntüyü aydınlatır (açık tonlara doğru iter).$\mathbf{\gamma > 1.0}$ (Örn: $2.0$): Görüntüyü karartır (koyu tonlara doğru iter).

In [ ]:
# Gerekli kütüphaneleri içe aktar
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt

# --- 1. train_df Yeniden Yükleme ---
# (Önceki adımlarda kullandığınız path'i kullanın)
BASE_DIR = '/kaggle/input/skin-cancer9-classesisic/'
image_paths = []
for dirname, _, filenames in os.walk(BASE_DIR):
    for filename in filenames:
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_paths.append(os.path.join(dirname, filename))

train_df = pd.DataFrame({'filepath': image_paths})
train_df = train_df[train_df['filepath'].str.contains('Train', case=False, na=False)].reset_index(drop=True)


# --- 2. Gamma Düzeltme Fonksiyonu Tanımlama ---
def adjust_gamma(image, gamma=1.0):
    # Düzeltme tablosunu oluştur (Look-Up Table - LUT)
    # Formül: output = 255 * (input / 255)^gamma
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255
                      for i in np.arange(256)]).astype("uint8")

    # LUT'u görüntüye uygula
    return cv2.LUT(image, table)

# --- 3. Görüntü Seçimi ve Yükleme ---
# 2.3/3.1'de analiz edilen Görüntü No 6'yı seçelim (Index 5)
random_samples = train_df.sample(n=9, random_state=42).reset_index(drop=True)
filepath = random_samples.loc[5, 'filepath']

# Yükleme (BGR formatında)
img_bgr_original = cv2.imread(filepath)
img_rgb_original = cv2.cvtColor(img_bgr_original, cv2.COLOR_BGR2RGB)
img_gray_original = cv2.cvtColor(img_bgr_original, cv2.COLOR_BGR2GRAY)

# --- 4. Gamma Değerlerini Uygulama ---
gamma_values = [0.5, 1.0, 2.0]
results = {}

for g in gamma_values:
    # RGB için uygulama
    results[f'RGB {g}'] = adjust_gamma(img_rgb_original, g)
    # Grayscale için uygulama
    results[f'GRAY {g}'] = adjust_gamma(img_gray_original, g)

# --- 5. Karşılaştırmalı Görselleştirme ---
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
fig.suptitle('3.3 Gamma Düzeltme Karşılaştırması (Gamma: 0.5, 1.0, 2.0)', fontsize=16)

# RGB Satırı
for i, g in enumerate(gamma_values):
    axes[0, i].imshow(results[f'RGB {g}'])
    axes[0, i].set_title(f'RGB (γ={g})')
    axes[0, i].axis('off')

# Grayscale Satırı
for i, g in enumerate(gamma_values):
    axes[1, i].imshow(results[f'GRAY {g}'], cmap='gray')
    axes[1, i].set_title(f'GRAY (γ={g})')
    axes[1, i].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

#

Gamma düzeltme işlemi, tek temsilci görüntü üzerinde $\mathbf{0.5}$, $\mathbf{1.0}$ ve $\mathbf{2.0}$ değerleriyle hem RGB hem de Grayscale formatlarına uygulanmıştır.1. Parlaklık Değişimlerinin Gözlemlenmesiγ = 1.0 (Referans): Hem RGB hem de Grayscale görüntüler değişmeden kalmıştır. Bu, düzeltme uygulanmayan orijinal parlaklık durumunu temsil eder.γ = 0.5 (Aydınlatma): $\mathbf{\gamma < 1.0}$ olduğu için, görüntüler belirgin şekilde aydınlanmıştır. Özellikle lezyonun koyu bölgeleri de dahil olmak üzere, görüntünün orta tonları açığa çıkmıştır. Bu, düşük ışık koşullarında çekilen veya gölgeli alanları baskın olan görüntülerdeki detayları ortaya çıkarmak için etkilidir.γ = 2.0 (Karartma): $\mathbf{\gamma > 1.0}$ olduğu için, görüntüler belirgin şekilde kararmıştır. Parlak bölgeler koyu tonlara doğru itilmiştir. Bu, görüntünün aşırı pozlanmış (overexposed) kısımlarındaki veya parlak tonlardaki detayları geri kazanmak için kullanılabilir.2. Görüntü Kalitesi Üzerindeki EtkisiKontrast Kontrolü: Gamma düzeltme, Histogram Eşitleme gibi tüm piksel dağılımını agresifçe değiştirmek yerine, sadece orta parlaklık tonlarını etkileyerek daha yumuşak ve doğal bir parlaklık kontrolü sağlar.Detay Kaybı Riski:$\mathbf{\gamma = 0.5}$ uygulamasında (Aydınlatma), görüntünün en parlak alanlarında (cilt arka planı) detay kaybı (saturasyon) riski oluşmuştur. Bu, çok parlak piksellerin $\mathbf{255}$'e yığılması anlamına gelir.$\mathbf{\gamma = 2.0}$ uygulamasında (Karartma), görüntünün en koyu alanlarında gölge detaylarının kaybolması (siyaha yığılması) riski gözlemlenmiştir.Sonuç: Gamma düzeltme, görüntü kalitesini düzeltmekten çok, görüntünün algısal parlaklığını belirli bir hedefe (daha parlak görünüm) ayarlamak için kullanılan, hassas bir araçtır. Özellikle cilt lezyonlarında koyu bölgelerin detaylarını kaybetmeden aydınlatmak ($\gamma < 1$) için faydalıdır.

# **4. Gürültü Azaltma**

In [ ]:
# Gerekli kütüphaneleri içe aktar
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt

# --- 1. train_df Yeniden Yükleme (Önceki Adımlardan Alındı) ---
BASE_DIR = '/kaggle/input/skin-cancer9-classesisic/'
image_paths = []
for dirname, _, filenames in os.walk(BASE_DIR):
    for filename in filenames:
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_paths.append(os.path.join(dirname, filename))

train_df = pd.DataFrame({'filepath': image_paths})
train_df = train_df[train_df['filepath'].str.contains('Train', case=False, na=False)].reset_index(drop=True)

# --- 2. Görüntü Seçimi ve Yükleme ---
# Temsilci görüntü (Index 5)
random_samples = train_df.sample(n=9, random_state=42).reset_index(drop=True)
filepath = random_samples.loc[5, 'filepath']

# Yükleme (BGR formatında)
img_bgr_original = cv2.imread(filepath)
img_rgb_original = cv2.cvtColor(img_bgr_original, cv2.COLOR_BGR2RGB)
img_gray_original = cv2.cvtColor(img_bgr_original, cv2.COLOR_BGR2GRAY)

# Filtre çekirdek boyutu (Kernel Size). Tek ve pozitif olmalıdır.
# 5x5 çekirdek kullanmak, etkiyi gözlemlemek için iyi bir boyuttur.
KERNEL_SIZE = 5

# --- 4.1. Median Blur Uygulama ---
# Median Blur, gürültüyü azaltmak için çekirdek içindeki piksellerin medyan değerini kullanır.
img_rgb_median = cv2.medianBlur(img_rgb_original, KERNEL_SIZE)
img_gray_median = cv2.medianBlur(img_gray_original, KERNEL_SIZE)


# --- 4.2. Gaussian Blur Uygulama ---
# Gaussian Blur, gürültüyü azaltmak için piksellerin ağırlıklı ortalamasını (Gauss fonksiyonu ile) kullanır.
# (5, 5) çekirdek boyutu ve 0 standart sapma (sigmaX=0, cv2 otomatik hesaplar)
img_rgb_gaussian = cv2.GaussianBlur(img_rgb_original, (KERNEL_SIZE, KERNEL_SIZE), 0)
img_gray_gaussian = cv2.GaussianBlur(img_gray_original, (KERNEL_SIZE, KERNEL_SIZE), 0)


# --- 5. Karşılaştırmalı Görselleştirme ---
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
fig.suptitle(f'4. Gürültü Azaltma Karşılaştırması (Kernel: {KERNEL_SIZE}x{KERNEL_SIZE})', fontsize=16)

# Satır 1: RGB Karşılaştırması
axes[0, 0].imshow(img_rgb_original)
axes[0, 0].set_title('Orijinal RGB')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_rgb_median)
axes[0, 1].set_title('RGB Median Blur')
axes[0, 1].axis('off')

axes[0, 2].imshow(img_rgb_gaussian)
axes[0, 2].set_title('RGB Gaussian Blur')
axes[0, 2].axis('off')

# Satır 2: Grayscale Karşılaştırması
axes[1, 0].imshow(img_gray_original, cmap='gray')
axes[1, 0].set_title('Orijinal Grayscale')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_gray_median, cmap='gray')
axes[1, 1].set_title('Grayscale Median Blur')
axes[1, 1].axis('off')

axes[1, 2].imshow(img_gray_gaussian, cmap='gray')
axes[1, 2].set_title('Grayscale Gaussian Blur')
axes[1, 2].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

#

Uygulanan her iki filtreleme yöntemi de (Kernel: $5\times5$) hem RGB hem de Grayscale görüntülerdeki ince doku gürültüsünü başarılı bir şekilde azaltmıştır . Farklar, özellikle keskinlik (edge) ve detay (detail) koruma noktalarında ortaya çıkmıştır.

**1. Median Blur Kenarları Daha İyi Korur mu?**

Evet,Median Blur uygulanmış görüntülerde, lezyonun sınırları ve yüzeydeki ince kıl (hair) yapıları, Gaussian Blur uygulanmış görüntülere kıyasla belirgin şekilde daha keskin kalmıştır.

Neden: Median filtresi doğrusal olmayan (non-lineer) bir işlemdir; gürültüyü aykırı değer olarak görerek medyanı kullanır ve bu sayede parlaklıkta ani sıçramalar olan kenarları etkili bir şekilde korur.

**2. Gaussian Blur Detay Kaybına Neden Oluyor mu?**

Evet,Gaussian Blur uygulanmış görüntülerde, lezyonun sınırları, kıl yapıları ve ince doku detayları belirgin bir şekilde yumuşatılmış (bulanıklaştırılmış) ve kaybolmuştur.

Neden: Gaussian filtresi doğrusal (lineer) bir işlemdir ve piksellerin ağırlıklı ortalamasını alarak çalışır. Bu ortalama alma işlemi, gürültüyle birlikte lezyonun ince (yüksek frekanslı) detaylarını da ortadan kaldırır.

Median Blur, cilt lezyonu analizinde (özellikle segmentasyon öncesinde) gürültüyü azaltırken lezyonun morfolojik sınırlarını ve şeklini korumak istediğimiz durumlarda Gaussian Blur'dan daha uygun bir filtredir.

Gaussian Blur ise, genel bir yumuşatma ve hazırlık filtresi olarak, özellikle uzamsal türev tabanlı işlemler (örneğin Canny kenar tespiti) öncesinde gürültünün etkisini azaltmak için tercih edilebilir, ancak detay kaybı riskini beraberinde getirir.

# **5. Döndürme ve Ayna Çevirme (Flipping)**

## **5.1. Rastgele Döndürme (Rotation)**

In [ ]:
# Gerekli kütüphaneleri içe aktar
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import random # Rastgele açı için

# --- 1. train_df Yeniden Yükleme ---
# (Önceki adımlarda kullandığınız path'i kullanın)
BASE_DIR = '/kaggle/input/skin-cancer9-classesisic/'
image_paths = []
for dirname, _, filenames in os.walk(BASE_DIR):
    for filename in filenames:
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_paths.append(os.path.join(dirname, filename))

train_df = pd.DataFrame({'filepath': image_paths})
train_df = train_df[train_df['filepath'].str.contains('Train', case=False, na=False)].reset_index(drop=True)

# --- 2. Görüntü Seçimi ve Yükleme ---
# Temsilci görüntü (Index 5)
random_samples = train_df.sample(n=9, random_state=42).reset_index(drop=True)
filepath = random_samples.loc[5, 'filepath']

# Yükleme (BGR formatında)
img_bgr_original = cv2.imread(filepath)
img_rgb_original = cv2.cvtColor(img_bgr_original, cv2.COLOR_BGR2RGB)
img_gray_original = cv2.cvtColor(img_bgr_original, cv2.COLOR_BGR2GRAY)


# --- 3. Rastgele Döndürme İşlemi ---
# 0 ile 10 derece arasında rastgele açı seçimi
angle = random.uniform(0, 10)
print(f"Rastgele Seçilen Döndürme Açısı: {angle:.2f} derece")

# Görüntü boyutları
(h, w) = img_rgb_original.shape[:2]
center = (w // 2, h // 2)

# Döndürme matrisini (M) oluşturma
# center: döndürme merkezi, angle: açı, 1.0: ölçek (scale)
M = cv2.getRotationMatrix2D(center, angle, 1.0)

# Döndürme matrisini uygulama
img_rgb_rotated = cv2.warpAffine(img_rgb_original, M, (w, h), borderValue=(255, 255, 255))
img_gray_rotated = cv2.warpAffine(img_gray_original, M, (w, h), borderValue=255) # Grayscale için sadece 255 (beyaz)

**5.2. Yatay Ayna Çevirme (Flipping) **

In [ ]:
# Ayna çevirme kodu (cv2.flip)
# 1: Yatay Ayna Çevirme (Horizontal Flip)
img_rgb_flipped = cv2.flip(img_rgb_original, 1)
img_gray_flipped = cv2.flip(img_gray_original, 1)

# --- 4. Karşılaştırmalı Görselleştirme ---
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
fig.suptitle(f'5. Geometrik Dönüşümler (Açı: {angle:.2f}°)', fontsize=16)

# Satır 1: RGB Karşılaştırması
axes[0, 0].imshow(img_rgb_original)
axes[0, 0].set_title('Orijinal RGB')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_rgb_rotated)
axes[0, 1].set_title(f'RGB Döndürme ({angle:.2f}°)')
axes[0, 1].axis('off')

axes[0, 2].imshow(img_rgb_flipped)
axes[0, 2].set_title('RGB Yatay Flip')
axes[0, 2].axis('off')

# Satır 2: Grayscale Karşılaştırması
axes[1, 0].imshow(img_gray_original, cmap='gray')
axes[1, 0].set_title('Orijinal Grayscale')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_gray_rotated, cmap='gray')
axes[1, 1].set_title(f'Grayscale Döndürme ({angle:.2f}°)')
axes[1, 1].axis('off')

axes[1, 2].imshow(img_gray_flipped, cmap='gray')
axes[1, 2].set_title('Grayscale Yatay Flip')
axes[1, 2].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

#

**5.1. Rastgele Döndürme (Rotation)**
Bu adımda, temsilci görüntü $0^\circ$ ile $10^\circ$ arasında rastgele seçilen bir açıda (Örn: $\sim 7.5^\circ$) döndürülmüştür.

Geometrik Etki: Döndürme işlemi, lezyonun şeklini ve piksel yoğunluk değerlerini korumuştur, ancak görüntünün dikey eksene göre yönelimini değiştirmiştir.

Kenar Etkisi: cv2.warpAffine fonksiyonu kullanıldığı için, görüntü döndürüldüğünde orijinal dikdörtgen alana tam olarak sığmamış ve ortaya çıkan yeni köşelerde boş pikseller (beyaz dolgu/padding) oluşmuştur. Bu, döndürme işleminin kaçınılmaz bir yan etkisidir ve bu dolgu piksellerinin analize dahil edilmeden önce kırpılması gerekebilir.

RGB ve Grayscale Karşılaştırması: Dönüşüm tamamen geometrik olduğu için, RGB ve Grayscale görüntüler üzerinde aynı matris operasyonu uygulanmıştır ve sonuç olarak her iki format da aynı uzamsal etkiye sahiptir.

**5.2. Yatay Ayna Çevirme (Flipping)**

Görüntü, dikey eksen etrafında yatay olarak ayna çevirme işlemine tabi tutulmuştur.

Geometrik Etki: Görüntü, sağdan sola doğru yansıtılmıştır. Bu işlem, görüntünün tüm piksel değerlerini korur, yalnızca yatay matris indekslerini tersine çevirir.
Görüntülerin Simetri Farkları Gözlemlenmesi:

Cilt lezyonları (özellikle melanom gibi kanserli lezyonlar) genellikle asimetriktir. Orijinal görüntü ile yatay flip yapılmış görüntünün birebir eşleşmemesi, lezyonun asimetri özelliğini açıkça gösterir.

Bu geometrik dönüşüm, lezyonun iki yarısının birbirine ne kadar benzediğini görsel olarak karşılaştırmamıza olanak tanır ve tanısal özelliklerden biri olan asimetriyi vurgular.

RGB ve Grayscale Karşılaştırması: Yatay çevirme, piksel değerlerini değil, yalnızca koordinatları etkilediği için, RGB ve Grayscale formatlarında aynı çevrilmiş ayna görüntüsü elde edilmiştir.

# **6. Frekans Alanında Filtreleme (FFT)**

In [ ]:
# Gerekli kütüphaneleri içe aktar
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt

# --- 1. train_df Yeniden Yükleme ---
# (Önceki adımlarda kullandığınız path'i kullanın)
BASE_DIR = '/kaggle/input/skin-cancer9-classesisic/'
image_paths = []
for dirname, _, filenames in os.walk(BASE_DIR):
    for filename in filenames:
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_paths.append(os.path.join(dirname, filename))

train_df = pd.DataFrame({'filepath': image_paths})
train_df = train_df[train_df['filepath'].str.contains('Train', case=False, na=False)].reset_index(drop=True)


# --- 2. Görüntü Seçimi ve Hazırlık ---
# Temsilci görüntü (Index 5)
random_samples = train_df.sample(n=9, random_state=42).reset_index(drop=True)
filepath = random_samples.loc[5, 'filepath']

# Yükleme (BGR formatında) ve Grayscale'e dönüştürme
img_bgr_original = cv2.imread(filepath)
# Görüntü işleme biterken RGB kullanılır, FFT için BGR'dan doğrudan Grayscale'e dönüştürelim
img_gray_original = cv2.cvtColor(img_bgr_original, cv2.COLOR_BGR2GRAY)


# --- 6.1. Fourier Dönüşümü ve Frekans Spektrumu ---

# Görüntü boyutlarını al
rows, cols = img_gray_original.shape

# Görüntünün float32 tipine dönüştürülmesi (FFT için gereklidir)
# Hata düzeltildi: cv2.DFT_COMPLEX_OUTPUT argümanı np.float32'den kaldırıldı
img_float32 = np.float32(img_gray_original)

# DFT (Discrete Fourier Transform) hesaplama
# cv2.DFT_COMPLEX_OUTPUT argümanı doğru yere, cv2.dft'ye eklendi
dft = cv2.dft(img_float32, flags=cv2.DFT_COMPLEX_OUTPUT)

# DFT spektrumunda merkez kaydırma (Düşük frekansları merkeze taşıma)
dft_shift = np.fft.fftshift(dft)

# Genlik spektrumunu hesaplama (Görselleştirme için)
# magnitude = log(1 + sqrt(real^2 + imag^2))
magnitude_spectrum = 20 * np.log(cv2.magnitude(dft_shift[:, :, 0], dft_shift[:, :, 1]) + 1)


# --- 6.2. Alçak Geçiren Filtre Uygulama (Low Pass Filter - LPF) ---

# Çekirdek boyutu (Filtrenin merkez genişliği). Etkiyi görmek için büyük bir çekirdek seçelim.
LPF_RADIUS = 30

# Maske oluşturma: Merkezde 1 (beyaz), kenarlarda 0 (siyah)
mask = np.zeros((rows, cols, 2), np.uint8)
center_row, center_col = rows // 2, cols // 2

# Dikdörtgen maske oluşturma (Alçak Geçiren Filtre)
mask[center_row - LPF_RADIUS:center_row + LPF_RADIUS,
      center_col - LPF_RADIUS:center_col + LPF_RADIUS] = 1

# Maskeyi frekans uzayına uygulama
fshift = dft_shift * mask


# --- 6.3. Ters Fourier Dönüşümü (Inverse FFT) ---

# Merkez kaydırmayı tersine çevirme
f_ishift = np.fft.ifftshift(fshift)

# Ters DFT (IDFT) hesaplama
img_back = cv2.idft(f_ishift)

# Ters DFT sonucunun genliğini alma ve 0-255 aralığına dönüştürme
img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])
img_back = cv2.normalize(img_back, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)


# --- 7. Karşılaştırmalı Görselleştirme ---
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 6))
fig.suptitle('6. Frekans Alanında Filtreleme (Alçak Geçiren Filtre) Analizi', fontsize=16)

# Sol: Orijinal Grayscale
axes[0].imshow(img_gray_original, cmap='gray')
axes[0].set_title('Orijinal Grayscale Görüntü')
axes[0].axis('off')

# Orta: Frekans Spektrumu (6.1)
axes[1].imshow(magnitude_spectrum, cmap='gray')
axes[1].set_title('Frekans Spektrumu (FFT)')
axes[1].axis('off')

# Sağ: Filtrelenmiş Uzamsal Görüntü (6.3)
axes[2].imshow(img_back, cmap='gray')
axes[2].set_title(f'Ters Fourier (Alçak Geçiren, R={LPF_RADIUS})')
axes[2].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

**1. Frekans Spektrumu (FFT) Analizi**
Enerji Yoğunlaşması: Frekans spektrumunun (orta görsel) tam merkezindeki parlak ve yoğun nokta (DC bileşeni), görüntünün enerjisinin büyük bir çoğunluğunun Düşük Frekanslarda toplandığını göstermektedir. Bu, cilt lezyonu görüntülerinin genellikle yavaş değişen büyük yapılardan (cilt rengi, lezyonun ana gövdesi) oluştuğu anlamına gelir.

Yüksek Frekanslar: Spektrumun kenarlarına doğru hızla azalan enerji, yüksek frekanslı bileşenlerin (keskin kenarlar, kıl, gürültü) görüntünün genel yapısına göre daha zayıf olduğunu teyit eder.

<table style="width:100%; border-collapse: collapse; font-family: Arial, sans-serif;">
    <thead>
        <tr style="background-color: #e6e6fa;">
            <th style="border: 1px solid #7a52a3; padding: 10px; text-align: left; color: #000;">Özellik</th>
            <th style="border: 1px solid #7a52a3; padding: 10px; text-align: left; color: #000;">Orijinal Grayscale (Uzamsal Alan)</th>
            <th style="border: 1px solid #7a52a3; padding: 10px; text-align: left; color: #000;">Ters Fourier Sonucu (LPF Uygulanmış)</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Görünüm</b></td>
            <td style="border: 1px solid #ddd; padding: 8px;">Keskin kenarlar, ince detaylar ve gürültü içerir.</td>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Bulanıklaşmış (Smoothed)</b> ve yumuşatılmış görünmektedir.</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Frekans Bilgisi</b></td>
            <td style="border: 1px solid #ddd; padding: 8px;">Yüksek ve Düşük frekanslı tüm bilgiyi içerir.</td>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Yüksek frekanslar sıfırlandığı için</b> sadece Düşük Frekanslı (görüntünün ana yapıları, yumuşak geçişler) bilgiyi içerir.</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Farkın Yorumu</b></td>
            <td style="border: 1px solid #ddd; padding: 8px;">—</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Alçak Geçiren Filtre (LPF) uygulaması, görüntüdeki $\mathbf{detay}$ ve $\mathbf{gürültü}$ gibi yüksek frekanslı bileşenlerin başarıyla baskılandığını gösteren matematiksel bir yumuşatma etkisidir.</td>
        </tr>
    </tbody>
</table>

```
# Bu, kod olarak biçimlendirilmiştir
```


# **7. Keskinleştirme ve Enterpolasyon**

**7.1. Unsharp Masking ile Keskinleştirme**

In [ ]:
# Gerekli kütüphaneleri içe aktar
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt

# --- 1. train_df Yeniden Yükleme ---
# (Önceki adımlarda kullandığınız path'i kullanın)
BASE_DIR = '/kaggle/input/skin-cancer9-classesisic/'
image_paths = []
for dirname, _, filenames in os.walk(BASE_DIR):
    for filename in filenames:
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_paths.append(os.path.join(dirname, filename))

train_df = pd.DataFrame({'filepath': image_paths})
train_df = train_df[train_df['filepath'].str.contains('Train', case=False, na=False)].reset_index(drop=True)


# --- 2. Görüntü Seçimi ve Yükleme ---
# Temsilci görüntü (Index 5)
random_samples = train_df.sample(n=9, random_state=42).reset_index(drop=True)
filepath = random_samples.loc[5, 'filepath']

# Yükleme (BGR formatında)
img_bgr_original = cv2.imread(filepath)
img_rgb_original = cv2.cvtColor(img_bgr_original, cv2.COLOR_BGR2RGB)
img_gray_original = cv2.cvtColor(img_bgr_original, cv2.COLOR_BGR2GRAY)


# --- 3. Unsharp Masking Uygulama ---

# Ayarlar
KERNEL_SIZE = 5     # Bulanıklaştırma çekirdek boyutu
ALPHA = 2.0         # Keskinleştirme ağırlığı (1.0'dan büyük olması güçlü keskinleştirme sağlar)

# Görüntüleri float'a dönüştürme (Matris çıkarma/ekleme işlemleri için gereklidir)
img_rgb_float = img_rgb_original.astype(np.float32)
img_gray_float = img_gray_original.astype(np.float32)


# --- RGB Unsharp Masking ---
# 1. Bulanıklaştırma (Düşük Frekans)
img_rgb_blurred = cv2.GaussianBlur(img_rgb_float, (KERNEL_SIZE, KERNEL_SIZE), 0)

# 2. Maske Hesaplama (Yüksek Frekans): Mask = Original - Blurred
mask_rgb = img_rgb_float - img_rgb_blurred

# 3. Keskinleştirme: Sharpened = Original + Alpha * Mask
img_rgb_sharpened = img_rgb_float + ALPHA * mask_rgb

# Sonucu uint8 tipine geri dönüştür ve 0-255 aralığında sınırla
img_rgb_sharpened = np.clip(img_rgb_sharpened, 0, 255).astype(np.uint8)


# --- Grayscale Unsharp Masking ---
# 1. Bulanıklaştırma (Düşük Frekans)
img_gray_blurred = cv2.GaussianBlur(img_gray_float, (KERNEL_SIZE, KERNEL_SIZE), 0)

# 2. Maske Hesaplama (Yüksek Frekans): Mask = Original - Blurred
mask_gray = img_gray_float - img_gray_blurred

# 3. Keskinleştirme: Sharpened = Original + Alpha * Mask
img_gray_sharpened = img_gray_float + ALPHA * mask_gray

# Sonucu uint8 tipine geri dönüştür ve 0-255 aralığında sınırla
img_gray_sharpened = np.clip(img_gray_sharpened, 0, 255).astype(np.uint8)


# --- 4. Karşılaştırmalı Görselleştirme ---
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
fig.suptitle(f'7.1 Unsharp Masking ile Keskinleştirme (Alpha: {ALPHA})', fontsize=16)

# RGB Karşılaştırması
axes[0, 0].imshow(img_rgb_original)
axes[0, 0].set_title('Orijinal RGB')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_rgb_sharpened)
axes[0, 1].set_title('Keskinleştirilmiş RGB')
axes[0, 1].axis('off')

# Grayscale Karşılaştırması
axes[1, 0].imshow(img_gray_original, cmap='gray')
axes[1, 0].set_title('Orijinal Grayscale')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_gray_sharpened, cmap='gray')
axes[1, 1].set_title('Keskinleştirilmiş Grayscale')
axes[1, 1].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

#

Keskinleştirme işlemi, düşük frekanslı bileşenleri (bulanık versiyon) orijinal görüntüden çıkarıp, geriye kalan yüksek frekanslı maskeyi ($\mathbf{\alpha=2.0}$ ağırlıkla) orijinal görüntüye ekleyerek uygulanmıştır.
1. Keskinleştirmenin Genel Etkisi ve Kalite

Netlik Artışı: Görüntünün genel netliği (perceived sharpness) belirgin şekilde artmıştır. Bu artış, lezyonun dış sınırlarında ve yüzeydeki ince kıl yapılarında en çok gözlemlenir. Bu etki, görüntünün $\mathbf{detay}$ (yüksek frekans) bilgisinin güçlendirildiğini gösterir.

Kenar Vurgusu (Overshoot): Kullanılan yüksek $\mathbf{\alpha=2.0}$ ağırlığı nedeniyle, özellikle koyu ve açık bölgeler arasındaki geçişlerde (kenarlarda), piksel yoğunluklarında ani ve güçlü sıçramalar (aşırı parlaklık veya aşırı koyuluk) meydana gelmiştir. Bu durum, keskinleştirmenin ticari yazılımlardaki karakteristik özelliğidir ve algılanan netliği artırır, ancak teknik olarak yapay gürültü ekler.

Gürültü Yükseltme: USM, görüntüdeki gürültü de yüksek frekanslı bir bileşen olduğu için, görüntünün arka planındaki rastgele gürültüyü de bir miktar güçlendirmiştir.


<table style="width:100%; border-collapse: collapse; font-family: Arial, sans-serif;">
    <thead>
        <tr style="background-color: #f7f7e0;">
            <th style="border: 1px solid #c9c99e; padding: 10px; text-align: left; color: #000;">Özellik</th>
            <th style="border: 1px solid #c9c99e; padding: 10px; text-align: left; color: #000;">Keskinleştirilmiş RGB (2x Bicubic)</th>
            <th style="border: 1px solid #c9c99e; padding: 10px; text-align: left; color: #000;">Keskinleştirilmiş Grayscale (2x Bicubic)</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Büyütme Yöntemi</b></td>
            <td colspan="2" style="border: 1px solid #ddd; padding: 8px; text-align: center;">Bicubic Enterpolasyon (En yakın 4x4 komşuluk kullanarak pürüzsüz büyütme sağlar)</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Temel Etki</b></td>
            <td style="border: 1px solid #ddd; padding: 8px;">Keskinleştirilmiş renkli görüntü, renk geçişlerini de pürüzsüzleştirerek büyür. Renk ve detay kalitesi yüksektir.</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Keskinleştirilmiş parlaklık bilgisi büyür. Bloklanma (mozaikleşme) etkisi düşüktür.</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Algılanan Kalite</b></td>
            <td style="border: 1px solid #ddd; padding: 8px;">Önceki adımda uygulanan keskinleştirme, büyütme sırasında oluşan doğal bulanıklığı maskelediği için algılanan netlik yüksektir.</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Büyük boyutlarda bile lezyonun sınırları, renk bilgisi olmamasına rağmen Bicubic sayesinde pürüzsüz kalır.</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;"><b>Karşılaştırma</b></td>
            <td style="border: 1px solid #ddd; padding: 8px;">Büyütülmüş format, teşhis amaçlı analiz için gerekli olan zengin renkli doku bilgisini koruduğu için en ideal formatı sunar.</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Hızlı uzamsal analiz ve boyut artırma gereksinimleri için hesaplama açısından daha verimli, ancak tanısal değer açısından RGB'den düşüktür.</td>
        </tr>
    </tbody>
</table>
```



**7.2. Bicubic Enterpolasyon**

In [ ]:
# Gerekli kütüphaneleri içe aktar
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt

# --- 1. train_df Yeniden Yükleme ve Görüntü İşleme Öncesi ---
BASE_DIR = '/kaggle/input/skin-cancer9-classesisic/'
image_paths = []
for dirname, _, filenames in os.walk(BASE_DIR):
    for filename in filenames:
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_paths.append(os.path.join(dirname, filename))

train_df = pd.DataFrame({'filepath': image_paths})
train_df = train_df[train_df['filepath'].str.contains('Train', case=False, na=False)].reset_index(drop=True)

# Temsilci görüntü (Index 5)
random_samples = train_df.sample(n=9, random_state=42).reset_index(drop=True)
filepath = random_samples.loc[5, 'filepath']

# Yükleme ve Keskinleştirme (7.1 adımından tekrar)
img_bgr_original = cv2.imread(filepath)
img_rgb_original = cv2.cvtColor(img_bgr_original, cv2.COLOR_BGR2RGB)
img_gray_original = cv2.cvtColor(img_bgr_original, cv2.COLOR_BGR2GRAY)

# Basit Keskinleştirme: Median blur yerine Gaussian'ı daha hızlı hesaplamak için kullanalım
# Unsharp Masking işleminin sonuçlarını yeniden hesaplayalım:
KERNEL_SIZE = 5
ALPHA = 2.0
img_rgb_float = img_rgb_original.astype(np.float32)
img_gray_float = img_gray_original.astype(np.float32)

# RGB Keskinleştirme
img_rgb_blurred = cv2.GaussianBlur(img_rgb_float, (KERNEL_SIZE, KERNEL_SIZE), 0)
img_rgb_sharpened = np.clip(img_rgb_float + ALPHA * (img_rgb_float - img_rgb_blurred), 0, 255).astype(np.uint8)

# Grayscale Keskinleştirme
img_gray_blurred = cv2.GaussianBlur(img_gray_float, (KERNEL_SIZE, KERNEL_SIZE), 0)
img_gray_sharpened = np.clip(img_gray_float + ALPHA * (img_gray_float - img_gray_blurred), 0, 255).astype(np.uint8)


# --- 7.2. Bicubic Enterpolasyon Uygulama ---
SCALE_FACTOR = 2

# RGB Bicubic Enterpolasyon
# fx, fy: Yatay ve dikey ölçek faktörü (2 kat büyüme)
img_rgb_bicubic = cv2.resize(img_rgb_sharpened, None, fx=SCALE_FACTOR, fy=SCALE_FACTOR, interpolation=cv2.INTER_CUBIC)

# Grayscale Bicubic Enterpolasyon
img_gray_bicubic = cv2.resize(img_gray_sharpened, None, fx=SCALE_FACTOR, fy=SCALE_FACTOR, interpolation=cv2.INTER_CUBIC)


# --- 3. Karşılaştırmalı Görselleştirme ---
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
fig.suptitle('7.2 Bicubic Enterpolasyon ile 2 Kat Büyütme', fontsize=16)

# RGB Karşılaştırması
axes[0, 0].imshow(img_rgb_sharpened)
axes[0, 0].set_title('Keskinleştirilmiş Orijinal (1x)')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_rgb_bicubic)
axes[0, 1].set_title(f'RGB Bicubic (2x) - Boyut: {img_rgb_bicubic.shape[0]}x{img_rgb_bicubic.shape[1]}')
axes[0, 1].axis('off')

# Grayscale Karşılaştırması
axes[1, 0].imshow(img_gray_sharpened, cmap='gray')
axes[1, 0].set_title('Keskinleştirilmiş Orijinal (1x)')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_gray_bicubic, cmap='gray')
axes[1, 1].set_title(f'Grayscale Bicubic (2x) - Boyut: {img_gray_bicubic.shape[0]}x{img_gray_bicubic.shape[1]}')
axes[1, 1].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

#

**Bicubic Enterpolasyonun Görüntü Kalitesine Etkisi**

Yöntemin Üstünlüğü: Bicubic enterpolasyon, Bilinear gibi basit yöntemlere göre daha karmaşık ve daha yüksek kalitede büyütme sağlar. Yeni piksel değerlerini hesaplamak için komşu piksellerin $\mathbf{4\times4}$ komşuluğunu kullanır. Bu sayede, büyütülmüş görüntüdeki kenarlar ve eğriler nispeten pürüzsüz kalmıştır.

Ön İşlemin Katkısı: Enterpolasyon öncesinde uygulanan keskinleştirme (USM) sayesinde, Bicubic büyütme sırasında oluşabilecek doğal yumuşama (smoothing) etkisi maskelenmiş ve büyütülmüş görüntülerin detay algısı yüksek tutulmuştur.

Sonuç: Elde edilen Bicubic büyütme sonuçları, görüntüleri büyütmek gerektiğinde dahi yüksek görsel netliği korumak için bu yöntemin uygun olduğunu göstermektedir.